Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Emily Holcomb"
COLLABORATORS = ""

---

# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
# YOUR CODE HERE
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
MYTABLE='eholcomb' + '_data'

# Then, this code will automatically setup your 
import os
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2019.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2019'
password = 'SLUcor2019'
database = 'slucor2019'

conn = create_engine('mysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + ':' + 
                     str(port) + '/' + 
                     database)

In [2]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [14]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'

# YOUR CODE HERE
data = pd.read_csv(url)
data.to_sql(MYTABLE, conn, if_exists = 'replace')

In [15]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [16]:
# YOUR CODE HERE
population = pd.read_sql_query('SELECT * FROM population', conn)

population = population[-population['zipcode'].str.contains('X')]
population['zipcode'] = pd.to_numeric(population['zipcode'])

datpop = data.merge(population,how='left',left_on='Zip_Code',right_on='zipcode')

In [17]:
datpop[datpop['population'] == datpop['population'].max()]

,X,Y,Provider_Name,NPI,CCN,Medicaid_EP_Hospital_Type,Street_Address,City,County,State,...,Last_Program_Year,Last_Payment_Year,Last_Payment_Criteria,Most_Recent_Disbursement_Amount,LONGITUDE,LATITUDE,FID,zipcode,population,geo_id
277,-118.099176,33.912508,"CPH HOSPITAL MANAGEMENT, LLC",1194016923,50771,Acute Care Hospitals,13100 STUDEBAKER RD,NORWALK,Los Angeles,CA,...,2016,2018,MU,668012,-118.099176,33.912508,278,90650.0,309633.0,8600000US90650


In [18]:
data[data['Zip_Code'] == 90650]

,X,Y,Provider_Name,NPI,CCN,Medicaid_EP_Hospital_Type,Street_Address,City,County,State,...,Payment_Year_Number,Program_Type,Total_payments,Last_Program_Year,Last_Payment_Year,Last_Payment_Criteria,Most_Recent_Disbursement_Amount,LONGITUDE,LATITUDE,FID
277,-118.099176,33.912508,"CPH HOSPITAL MANAGEMENT, LLC",1194016923,50771,Acute Care Hospitals,13100 STUDEBAKER RD,NORWALK,Los Angeles,CA,...,2,Medicare/Medicaid,1781364,2016,2018,MU,668012,-118.099176,33.912508,278


In [19]:
answer = ['1194016923']

In [20]:
assert(type(answer) == list)


## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [21]:
# YOUR CODE HERE

grp = data.groupby(['Zip_Code','Medicaid_EP_Hospital_Type'],as_index = False).sum()

pdgrp = grp[['Zip_Code','Total_payments','Medicaid_EP_Hospital_Type']]

grppop = pdgrp.merge(population,how='left',left_on='Zip_Code',right_on='zipcode')

grppop['Avg_Pay_per_Capita'] = grppop['Total_payments']/grppop['population']

answer = grppop.groupby('Medicaid_EP_Hospital_Type',as_index=False).mean()

In [22]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(answer[answer['Medicaid_EP_Hospital_Type'] == 'Acute Care Hospitals']['Avg_Pay_per_Capita'].item() == 32.74548165)
assert(answer[answer['Medicaid_EP_Hospital_Type'] == 'Children\'s Hospitals']['Avg_Pay_per_Capita'].item() == 124.449290)


AssertionError: 

In [23]:
answer

,Medicaid_EP_Hospital_Type,Zip_Code,Total_payments,zipcode,population,Avg_Pay_per_Capita
0,Acute Care Hospitals,93223.010563,2.496904e+06,93250.26259,105039.971223,32.745483
1,Children's Hospitals,92706.090909,4.448898e+06,92613.10000,86585.400000,124.449290
